# Budowanie modeli - zad 5_2

### Generowanie zbioru treningowego i walidacyjnego

In [29]:
from pandas.core.arrays import datetimes
import json
import itertools
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings('ignore')


def prepare_data(data_begin, data_end):
    events_data = pd.read_json("/content/drive/My Drive/IUM/sessions.jsonl", lines=True)
    products_data = pd.read_json("/content/drive/My Drive/IUM/products.jsonl", lines=True)

    events_data['year'] = events_data.timestamp.dt.year
    events_data['month'] = events_data.timestamp.dt.month

    years = list(events_data.year.unique())
    months = list(events_data.month.unique())
    user_ids = list(events_data.user_id.unique())

    triplets = []
    for triplet in itertools.product(years, months, user_ids):
        triplets.append(triplet)

    dates=pd.date_range(data_begin,data_end, 
              freq='MS').tolist()

  
    processed_data = pd.DataFrame(triplets, columns=['year', 'month', 'user_id'])
 

    buying_events_month = events_data[events_data['event_type'] == 'BUY_PRODUCT'].groupby(
        ['user_id', 'month', 'year']).aggregate({'session_id': 'count'}).rename(
        columns={'session_id': 'buying_events_month'}).reset_index()


    all_events_month = events_data.groupby(
    ['user_id', 'month', 'year']).aggregate({'session_id': 'count'}).rename(
    columns={'session_id': 'all_events_month'}).reset_index()

    for i, date in enumerate(dates):
      first_month, first_year = date.month, date.year
     
      buying_events_first_month = buying_events_month[
          (buying_events_month['month'] == first_month) & (buying_events_month['year'] == first_year)]

      if first_month == 12:
        second_month = 1
        second_year=first_year+1
      else:
        second_month = first_month + 1
        second_year=first_year

      buying_events_second_month = buying_events_month[
          (buying_events_month['month'] == second_month) & (buying_events_month['year'] == second_year)]

      ######## just to make merge easier
      buying_events_second_month['month'] = first_month
      buying_events_second_month['year'] = first_year

 
      all_events_first_month = all_events_month[
          (all_events_month['month'] == first_month) & (all_events_month['year'] == first_year)]
     
      all_events_second_month = all_events_month[
          (all_events_month['month'] == second_month) & (all_events_month['year'] == second_year)]


      ######## just to make merge easier
      all_events_second_month['month'] = first_month
      all_events_second_month['year'] = first_year

      df=pd.merge(buying_events_first_month, buying_events_second_month, how='left', on=['year', 'month', 'user_id'])#, 
  
      df=pd.merge(df, all_events_first_month, how='left', on=['year', 'month', 'user_id'])
      df=pd.merge(df, all_events_second_month, how='left', on=['year', 'month', 'user_id'])

      df.fillna(0,inplace=True)


      df['first_month_sin']=np.sin(df['month'] * (2. * np.pi / 12))
      df['first_month_cos']=np.cos(df['month'] * (2. * np.pi / 12))


      buying_sessions = events_data[events_data['event_type'] == 'BUY_PRODUCT']
  

      deals = pd.merge(buying_sessions, products_data[['product_id','price']], how='left', on=['product_id'])
      deals['final_price'] = deals['price']

      pd.set_option("display.max_rows", None, "display.max_columns", None)

      spent_money_first_month = deals[(deals.timestamp.dt.month == first_month) & (deals.timestamp.dt.year == first_year)].groupby('user_id').aggregate(  
          {'final_price': 'sum'}).rename(columns={'final_price': 'spent_money_first_month'}).reset_index()

      spent_money_second_month = deals[(deals.timestamp.dt.month == second_month) & (deals.timestamp.dt.year ==second_year)].groupby('user_id').aggregate(
          {'final_price': 'sum'}).rename(
          columns={'final_price': 'spent_money_second_month'}).reset_index()

      if second_month == 12:
        third_month = 1
        third_year=second_year+1
      else:
        third_month = second_month + 1
        third_year=second_year


      spent_money_third_month = deals[(deals.timestamp.dt.month == third_month) & (deals.timestamp.dt.year == third_year)].groupby('user_id').aggregate(
          {'final_price': 'sum'}).rename(
          columns={'final_price': 'spent_money_third_month'}).reset_index()

      spent_money_average = pd.merge(spent_money_first_month, spent_money_second_month, how='left', on=['user_id'])
      spent_money_average['spent_money_second_month'].fillna(0, inplace=True)
      spent_money_average['spent_money_total_average'] = spent_money_average[['spent_money_first_month',
                                                                              'spent_money_second_month']].sum(axis=1) / 2


      spent_money_average = pd.merge(spent_money_average, spent_money_third_month, how='left', on=['user_id'])
      spent_money_average['spent_money_third_month'].fillna(0, inplace=True)

      spent_money_average['label'] = np.where(spent_money_average['spent_money_third_month'] - spent_money_average['spent_money_total_average'] > 100, 1, 0)
      processed_data = pd.merge(df, spent_money_average[['spent_money_first_month', 'spent_money_second_month', 'user_id', 'label']], how='left', on=['user_id'])

      processed_data['label'].fillna(0, inplace=True)
      processed_data = processed_data.sort_values(by=['user_id']).reset_index()


      del processed_data['index']
      del processed_data['user_id']
      del processed_data['month']
 
      processed_data = processed_data.drop_duplicates(keep='first')
      #print(processed_data.shape)
      if i==0:
        dataset=processed_data
      else:
        dataset=pd.concat([dataset, processed_data])

    return dataset
      
df_train=prepare_data('2019.09.01', '2021.12.31')
df_train.to_csv('processed_data_train.csv')

In [30]:
df_val=prepare_data('2021.01.01', '2021.03.31')
df_val.to_csv('processed_data_val.csv')

Podział na zbiór treningowy i walidacyjny odbył się w sposób zgodny z dziedziną szeregów czasowych, tj. zbiór treningowy i walidacyjny mają rekordy z różnych chwil czasu. Jest to inne podejście niż w przypadku gdy całą ramkę danych dzieli się losowo w stosunku 70:30 albo 80:20.

In [32]:
print(df_train.shape, df_val.shape)

(5890, 10) (760, 10)


### Tworzenie modeli

In [3]:
import sklearn 

In [36]:
np.random.seed(0)

In [37]:
df_train.columns

Index(['year', 'buying_events_month_x', 'buying_events_month_y',
       'all_events_month_x', 'all_events_month_y', 'first_month_sin',
       'first_month_cos', 'spent_money_first_month',
       'spent_money_second_month', 'label'],
      dtype='object')

Wyodrębnienie zmiennych wejściowych do modelu i etykiet.

In [38]:
X_train, y_train = df_train.iloc[:,:-1], df_train.iloc[:,-1]
X_val, y_val = df_val.iloc[:,:-1], df_val.iloc[:,-1]

In [39]:
df_train['label'].mean()

0.2266553480475382

In [40]:
df_val['label'].mean()

0.2236842105263158

Zbiór treningowy i walidacyjny są wysoko niezbalansowane - mają tylko ok. 22% wartości pozytywnej w etykietach. Zdecydowaliśmy się nie dokonywać over/undersamplingu z uwagi na informację na wykładzie, że poziom niezbalansowania nawet na poziomie 10:90 pozwala na wytrenowanie modelu bez dodatkowych przekształceń balansujących. 

W tej sytuacji należy pamiętać, że model naiwny, który zawsze będzie zwracać wartość 0, będzie miał accuracy na poziomie ok. 77.6%. Dlatego też wartości accuracy większe od tego poziomu będą dopiero oznaczać, że model wnosi wartość dodaną.

W pierwszych próbach wytrenowania modeli, nie wykonaliśmy normalizacji danych, aby sprawdzić jaką jakość modelu jesteśmy w stanie uzyskac na danych surowych.

### Naive Bayes

In [41]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
model.fit(X_train, y_train)
model.score(X_train, y_train)

0.7733446519524618

In [42]:
model.score(X_val, y_val)

0.7763157894736842

In [45]:
model.predict(X_val).sum()

0

Model niestety zgodnie z jego nazwą okazał się naiwny - ma wartość accuracy odpowiadającą wartości bazowej. Model zwraca same wartości zero.

### Regresja logistyczna

In [84]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=0).fit(X_train, y_train)


model.score(X_train, y_train) 

0.7733446519524618

In [85]:
model.score(X_val, y_val) 

0.7763157894736842

Zgodnie z przewidywaniami, model regresji logistycznej też był zbyt prosty, aby wychwycić złożone zależności w danych.

In [86]:
import pickle
filename = 'logistic_regression_model.pickle'
pickle.dump(model, open(filename, 'wb'))

### Las losowy

In [78]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)
model.score(X_train, y_train)

0.997283531409168

In [79]:
model.score(X_val, y_val)

0.9960526315789474

Model lasu losowego osiaga bardzo wysoką skuteczność na zbiorze walidacyjnym - 99.6%. Dla zbiorów niezbalansowanych, często bada się też miarę AUC.

In [80]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_val, model.predict_proba(X_val)[:,1])

0.9999551345962114

Miara AUC na zbiorze walidacyjnym wynosi praktycznie 1 czyli tyle ile wartość maksymalna.

In [81]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_val, model.predict(X_val))

array([[590,   0],
       [  3, 167]])

Macierz pomyłek potwierdza, że stworzony model jest bardzo dobry. Pomylił się podczas predykcji jedynie 3 próbek.

Poniżej są dokładne parametry modelu. Z uwagi na to, że jego jakośc na zbiorze treningowym i walidacyjnym jest taka sama, można stwierdzić, że nie jest on przeuczony.

In [82]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}

In [83]:
import pickle
filename = 'random_forest_model.pickle'
pickle.dump(model, open(filename, 'wb'))

### Ciekawostka

Postanowiliśmy sprawdzić jaki model zostanie wskazany jako optymalny z użyciem algorytmów AutoML z pakietu TPOT i HyperOpt.

## TPOT

In [ ]:
!pip install tpot

In [12]:
from tpot import TPOTClassifier

tpot = TPOTClassifier(generations=3,verbosity=2)

tpot.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/400 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7733446519524618

Generation 2 - Current best internal CV score: 0.7735144312393887

Generation 3 - Current best internal CV score: 0.7736842105263158

Best pipeline: MLPClassifier(input_matrix, alpha=0.01, learning_rate_init=0.01)


TPOTClassifier(generations=3, verbosity=2)

In [13]:
tpot.score(X_train, y_train)

0.7733446519524618

In [14]:
tpot.score(X_val, y_val)

0.7763157894736842

Okazało się, że wskazany jako optymalny został model oparty na prostej sieci neuronowej typu perceptron wielowarstwowy. Niemniej jednak dokładność tego modelu odpowiada modelowi naiwnemu.

## HyperOpt

In [ ]:
!pip install hyperopt

In [ ]:
!pip install git+https://github.com/hyperopt/hyperopt-sklearn

In [22]:
from hpsklearn import HyperoptEstimator
from hpsklearn import any_classifier, any_preprocessing
from hyperopt import tpe

W ramach poszukiwania optymalnego rozwiązania, optymalizowany jest nie tylko wybór modelu ale także preprocessing.

In [27]:
model = HyperoptEstimator(classifier=any_classifier('cla'), preprocessing=any_preprocessing('pre'), algo=tpe.suggest, max_evals=10, trial_timeout=50000)

model.fit(X_train, y_train)
acc = model.score(X_val, y_val)
print("Accuracy: %.3f" % acc)
print(model.best_model())

100%|██████████| 10/10 [00:00<00:00,  3.55trial/s, best loss: 0.22920203735144318]
Accuracy: 0.776
{'learner': SVC(C=1.2612590724982196, coef0=0.40225014748053256,
    decision_function_shape='ovo', degree=5, gamma='auto', kernel='poly',
    random_state=4, tol=0.00032053579174632675), 'preprocs': (MinMaxScaler(feature_range=(-1.0, 1.0)),), 'ex_preprocs': ()}


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  "X does not have valid feature names, but"


Okazało się, że jako optymalny został wskazany SVM. Niemniej jednak dokładność tego modelu odpowiada modelowi naiwnemu.

# Podsumowanie

Algorytmy AUTOML nie potrafiły wytrenować modelu dobrej jakości w sytuacji niezbalansowania zbioru danych.
Finalnie najlepszy model stanowi model lasu losowego, który popełnił na zbiorze walidacyjnym tylko 3 pomyłki, co przełożyło się na prawie maksymalną możliwą miarę accuracy i AUC. 